In [23]:
# importing some python modules - just in case
import pandas as pd
import numpy as np
import os, sys 
import io
import ee

In [24]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [10]:
#setting up google drive connection
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# making sure pyspark is up to date
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Ign:13 https://developer.download.nvidia.com/comp

In [3]:
#pyspark
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [4]:
# import likely needed modules
from pyspark import SparkFiles
from pyspark import SparkConf                                                                                                                 
from pyspark.context import SparkContext                                                                                                      
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import avg
from pyspark.sql.functions import translate
from pyspark.sql.functions import desc

In [11]:
# from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

In [6]:
#code for remount if needed
#drive.mount("/content/drive", force_remount=True)

In [12]:
#csv location
path = "/content/drive/My Drive/data/clean_ppp_data.csv"

In [8]:
# pandas check
# df=pd.read_csv(path)
# df.sample()

In [9]:
# needed in order to do large file save 
# %pip install py4j


### Import csv

In [13]:
# csv into pyspark dataframe
df=spark.read.csv(SparkFiles.get(path), sep=",", header=True)
df.show()

+-----------+----------------+-----+-----+----------+--------------------+----------+------------+-----------+----------+----------+--------------------+
|Loan Amount|            City|State|  Zip|NAICS Code|       Business Type|      Race|      Gender|    Veteran|Jobs Saved|      Date|                Bank|
+-----------+----------------+-----+-----+----------+--------------------+----------+------------+-----------+----------+----------+--------------------+
|   149961.0|     KANSAS CITY|   MO|64108|     54199|Limited  Liabilit...|Unanswered|  Unanswered| Unanswered|      13.0|2020-04-13|   Country Club Bank|
|   149900.0|         RAYTOWN|   MO|64133|     44112|         Corporation|     White|  Male Owned| Unanswered|      14.0|2020-05-11|Blue Ridge Bank a...|
|   149900.0|     VALLEY PARK|   MO|63088|     62441|         Corporation|Unanswered|  Unanswered| Unanswered|      28.0|2020-04-09| Meramec Valley Bank|
|   149810.9|          ADRIAN|   MO|64720|     23811|Limited  Liabilit...|  

Check out the data and cast types if needed



In [14]:
# cast from int to string
df = df.withColumn("NAICS Code", df['NAICS Code'].cast('string'))


In [15]:
#remove " from NY - Sate column
from pyspark.sql.functions import udf
quoteReplace=udf(lambda x: x.replace(u'"',''))
df=df.withColumn('State',quoteReplace('State'))
dotReplace=udf(lambda x: x.replace(u'.',''))
df=df.withColumn('State',dotReplace('State'))
df.show(2,truncate=False)


+-----------+-----------+-----+-----+----------+-------------------------------+----------+----------+----------+----------+----------+-----------------------------+
|Loan Amount|City       |State|Zip  |NAICS Code|Business Type                  |Race      |Gender    |Veteran   |Jobs Saved|Date      |Bank                         |
+-----------+-----------+-----+-----+----------+-------------------------------+----------+----------+----------+----------+----------+-----------------------------+
|149961.0   |KANSAS CITY|MO   |64108|54199     |Limited  Liability Company(LLC)|Unanswered|Unanswered|Unanswered|13.0      |2020-04-13|Country Club Bank            |
|149900.0   |RAYTOWN    |MO   |64133|44112     |Corporation                    |White     |Male Owned|Unanswered|14.0      |2020-05-11|Blue Ridge Bank and Trust Co.|
+-----------+-----------+-----+-----+----------+-------------------------------+----------+----------+----------+----------+----------+-----------------------------+
only

In [16]:
# df.printSchema()

In [17]:
#create a general category column for NAICS codes

df=df.withColumn('NAICS_cat', df['NAICS Code'].substr(0, 2))
df.show(5)

+-----------+-------------+-----+-----+----------+--------------------+----------+------------+-----------+----------+----------+--------------------+---------+
|Loan Amount|         City|State|  Zip|NAICS Code|       Business Type|      Race|      Gender|    Veteran|Jobs Saved|      Date|                Bank|NAICS_cat|
+-----------+-------------+-----+-----+----------+--------------------+----------+------------+-----------+----------+----------+--------------------+---------+
|   149961.0|  KANSAS CITY|   MO|64108|     54199|Limited  Liabilit...|Unanswered|  Unanswered| Unanswered|      13.0|2020-04-13|   Country Club Bank|       54|
|   149900.0|      RAYTOWN|   MO|64133|     44112|         Corporation|     White|  Male Owned| Unanswered|      14.0|2020-05-11|Blue Ridge Bank a...|       44|
|   149900.0|  VALLEY PARK|   MO|63088|     62441|         Corporation|Unanswered|  Unanswered| Unanswered|      28.0|2020-04-09| Meramec Valley Bank|       62|
|   149810.9|       ADRIAN|   MO|6

In [ ]:
# df.describe()

In [18]:
gender=df.orderBy(df['Gender'].asc())
# gender.show()

**Quick look at veterans, men and women**

In [19]:
#women, men & veteran dataframes
womens = df.filter(df['Gender']== 'Female Owned')
mens = df.filter(df['Gender']== 'Male Owned')
vets=df.filter(df['Veteran']== 'Veteran')

In [20]:
vets.describe().show()

+-------+------------------+--------+-----+------------------+-----------------+-------------+--------------------+------------+-------+-----------------+----------+-------------------+------------------+
|summary|       Loan Amount|    City|State|               Zip|       NAICS Code|Business Type|                Race|      Gender|Veteran|       Jobs Saved|      Date|               Bank|         NAICS_cat|
+-------+------------------+--------+-----+------------------+-----------------+-------------+--------------------+------------+-------+-----------------+----------+-------------------+------------------+
|  count|             24727|   24727|24727|             24727|            24727|        24727|               24727|       24727|  24727|            24727|     24727|              24727|             24727|
|   mean| 45514.65032070205| 19962.0| null| 53077.57649532899|249829.6230031949|         null|                null|        null|   null|6.006672867715452|      null|               

In [21]:
vets.show()

+-----------+-----------+-----+-----+----------+--------------------+--------------------+------------+-------+----------+----------+--------------------+---------+
|Loan Amount|       City|State|  Zip|NAICS Code|       Business Type|                Race|      Gender|Veteran|Jobs Saved|      Date|                Bank|NAICS_cat|
+-----------+-----------+-----+-----+----------+--------------------+--------------------+------------+-------+----------+----------+--------------------+---------+
|   148437.5|SAINT LOUIS|   MO|63132|    541922|         Corporation|          Unanswered|  Male Owned|Veteran|      12.0|2020-04-10|     Carrollton Bank|       54|
|   147287.5|SPRINGFIELD|   MO|65803|     23821|         Corporation|          Unanswered|  Male Owned|Veteran|      11.0|2020-04-12|   BancorpSouth Bank|       23|
|   146673.0| HALLSVILLE|   MO|65255|     49211|         Corporation|          Unanswered|  Male Owned|Veteran|       2.0|2020-05-19|             Kabbage|       49|
|   145594

In [22]:
womens.describe().show()


+-------+-----------------+-------------------+------+------------------+------------------+-------------+--------------------+------------+-----------+-----------------+----------+----------------+------------------+
|summary|      Loan Amount|               City| State|               Zip|        NAICS Code|Business Type|                Race|      Gender|    Veteran|       Jobs Saved|      Date|            Bank|         NAICS_cat|
+-------+-----------------+-------------------+------+------------------+------------------+-------------+--------------------+------------+-----------+-----------------+----------+----------------+------------------+
|  count|           207583|             207583|207583|            207583|            207583|       207583|              207583|      207583|     207583|           207583|    207583|          207583|            207583|
|   mean|34935.94942013533|               null|  null| 52829.98377034728|271107.92484452005|         null|                null| 

In [ ]:
mens.describe().show()



In [23]:
df.select(avg("Loan Amount")).show()

+-----------------+
| avg(Loan Amount)|
+-----------------+
|33791.73115993187|
+-----------------+



In [24]:
gen_vets=df.groupBy('Gender', 'Veteran').count().orderBy('count')
gen_vets.show(5)

+------------+----------+-----+
|      Gender|   Veteran|count|
+------------+----------+-----+
|  Unanswered|Male Owned|    1|
|       White|Male Owned|    1|
|  Unanswered|   Veteran| 1344|
|Female Owned|   Veteran| 2810|
|  Male Owned|   Veteran|20573|
+------------+----------+-----+
only showing top 5 rows



In [25]:
vet_ct=df.groupBy('Veteran').count().orderBy('count')
vet_ct.show()

+-----------+-------+
|    Veteran|  count|
+-----------+-------+
| Male Owned|      2|
|    Veteran|  24727|
|Non-Veteran| 545615|
| Unanswered|3252814|
+-----------+-------+



In [26]:
rc_ct=df.groupBy('Race').count().orderBy('count', ascending=False)
rc_ct.show()

+--------------------+-------+
|                Race|  count|
+--------------------+-------+
|          Unanswered|3407771|
|               White| 319185|
|               Asian|  47993|
|            Hispanic|  32408|
|Black or African ...|  13513|
|American Indian o...|   2134|
|        Puerto Rican|    124|
|Non-Profit Organi...|     23|
|         Multi Group|      4|
|      Eskimo & Aleut|      3|
+--------------------+-------+



In [27]:
race=df.filter(df['Race'] != 'Unanswered')
race.show(5)


+-----------+-----------+-----+-----+----------+--------------------+-----+------------+-----------+----------+----------+--------------------+---------+
|Loan Amount|       City|State|  Zip|NAICS Code|       Business Type| Race|      Gender|    Veteran|Jobs Saved|      Date|                Bank|NAICS_cat|
+-----------+-----------+-----+-----+----------+--------------------+-----+------------+-----------+----------+----------+--------------------+---------+
|   149900.0|    RAYTOWN|   MO|64133|     44112|         Corporation|White|  Male Owned| Unanswered|      14.0|2020-05-11|Blue Ridge Bank a...|       44|
|   149810.9|     ADRIAN|   MO|64720|     23811|Limited  Liabilit...|White|Female Owned|Non-Veteran|      13.0|2020-04-13|         Adrian Bank|       23|
|   149797.5|SAINT LOUIS|   MO|63126|     48423|Limited  Liabilit...|White|  Male Owned|Non-Veteran|      12.0|2020-04-07|  1st Advantage Bank|       48|
|  149733.99|  ST. LOUIS|   MO|63144|    722511|         Corporation|White| 

In [28]:
people_df = df.filter(
    (df['Veteran'] != 'Unanswered') & 
    (df['Race'] != 'Unanswered') & 
    (df['Gender'] != 'Unanswered') 
    )
people_df.show(10)

+-----------+-------------+-----+-----+----------+--------------------+--------------------+------------+-----------+----------+----------+--------------------+---------+
|Loan Amount|         City|State|  Zip|NAICS Code|       Business Type|                Race|      Gender|    Veteran|Jobs Saved|      Date|                Bank|NAICS_cat|
+-----------+-------------+-----+-----+----------+--------------------+--------------------+------------+-----------+----------+----------+--------------------+---------+
|   149810.9|       ADRIAN|   MO|64720|     23811|Limited  Liabilit...|               White|Female Owned|Non-Veteran|      13.0|2020-04-13|         Adrian Bank|       23|
|   149797.5|  SAINT LOUIS|   MO|63126|     48423|Limited  Liabilit...|               White|  Male Owned|Non-Veteran|      12.0|2020-04-07|  1st Advantage Bank|       48|
|  149733.99|    ST. LOUIS|   MO|63144|    722511|         Corporation|               White|  Male Owned|Non-Veteran|      15.0|2020-04-08|Citize

In [29]:
rc_ct=people_df.groupBy('Race',).count().orderBy('count', ascending=False)
rc_ct.show(5)

+--------------------+------+
|                Race| count|
+--------------------+------+
|               White|208476|
|               Asian| 34888|
|            Hispanic| 22166|
|Black or African ...|  8565|
|American Indian o...|  1471|
+--------------------+------+
only showing top 5 rows



In [30]:
people_df.count()

275661

In [31]:
#number of rows  for loan counts per 'state' (probably includes virgin islands) from people dataframe
state_loan_count = people_df.groupby("State").agg({"Loan Amount": "count"})
state_loan_count.count()





58

In [32]:
# loan count per state from smaller dataset (people)
state_loan_count.show()

+-----+------------------+
|State|count(Loan Amount)|
+-----+------------------+
|   SC|              2676|
|   AZ|              3738|
|   LA|              3971|
|   MN|             10098|
|   NJ|              3791|
|   DC|               301|
|   OR|              1895|
|   VA|              4022|
|   RI|              1381|
|   WY|              1319|
|   KY|              3381|
|   NH|              1605|
|   MI|              8292|
|   NV|              1353|
|   WI|              7327|
|   ID|              1809|
|   CA|             22052|
|   CT|              4197|
|   NE|              5816|
|   MT|              2892|
+-----+------------------+
only showing top 20 rows



Send the "people" dataframe to csv.

In [33]:
# relatively small amount of rows, but looks like it will be interesting to work with 
# sending to file and will run visualizations in pandas/tableau and run machine learning on it

people_df.toPandas().to_csv("people_ppp.csv", header=True)
!cp people_ppp.csv "drive/My Drive/"

In [34]:
# banks in main dataframe
top_banks=df.groupBy("Bank").count().orderBy('count', ascending=False)
top_banks.show(10)

+--------------------+------+
|                Bank| count|
+--------------------+------+
|     Bank of America|286122|
| JPMorgan Chase Bank|197511|
|    Cross River Bank|122920|
|             Kabbage|113504|
|    Wells Fargo Bank|110070|
|           U.S. Bank| 82922|
|Celtic Bank Corpo...| 80472|
|             WebBank| 71945|
|Truist Bank d/b/a...| 61601|
|             TD Bank| 56340|
+--------------------+------+
only showing top 10 rows



In [35]:
# NAICS codes in main dataframe
biz_code=df.groupBy("NAICS Code").count().orderBy('count', ascending=False)
biz_code.show(10)

+----------+------+
|NAICS Code| count|
+----------+------+
|    722511|129239|
|     53121|103133|
|     54111|102117|
|     62121| 84500|
|    812112| 80222|
|    621111| 79396|
|     99999| 75184|
|     81311| 73448|
|     52421| 71632|
|    722513| 63869|
+----------+------+
only showing top 10 rows



In [36]:
# NAICS general categories in main dataframe
top_cat_count=df.groupBy("NAICS_cat").count().orderBy('count', ascending=False).show(10)


+---------+------+
|NAICS_cat| count|
+---------+------+
|       54|518202|
|       81|455829|
|       62|395165|
|       23|355120|
|       72|289437|
|       44|248358|
|       53|213868|
|       56|195950|
|       48|146207|
|       52|143450|
+---------+------+
only showing top 10 rows



In [37]:
#verifying nan value removal

from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+-----------+----+-----+---+----------+-------------+----+------+-------+----------+----+----+---------+
|Loan Amount|City|State|Zip|NAICS Code|Business Type|Race|Gender|Veteran|Jobs Saved|Date|Bank|NAICS_cat|
+-----------+----+-----+---+----------+-------------+----+------+-------+----------+----+----+---------+
|          0|   0|    0|  0|         0|            0|   0|     0|      0|         0|   0|   0|        0|
+-----------+----+-----+---+----------+-------------+----+------+-------+----------+----+----+---------+



In [38]:
df.show()

+-----------+----------------+-----+-----+----------+--------------------+----------+------------+-----------+----------+----------+--------------------+---------+
|Loan Amount|            City|State|  Zip|NAICS Code|       Business Type|      Race|      Gender|    Veteran|Jobs Saved|      Date|                Bank|NAICS_cat|
+-----------+----------------+-----+-----+----------+--------------------+----------+------------+-----------+----------+----------+--------------------+---------+
|   149961.0|     KANSAS CITY|   MO|64108|     54199|Limited  Liabilit...|Unanswered|  Unanswered| Unanswered|      13.0|2020-04-13|   Country Club Bank|       54|
|   149900.0|         RAYTOWN|   MO|64133|     44112|         Corporation|     White|  Male Owned| Unanswered|      14.0|2020-05-11|Blue Ridge Bank a...|       44|
|   149900.0|     VALLEY PARK|   MO|63088|     62441|         Corporation|Unanswered|  Unanswered| Unanswered|      28.0|2020-04-09| Meramec Valley Bank|       62|
|   149810.9|   

In [39]:
df.filter(df['NAICS_cat'].isNotNull()).count()


3823158

In [40]:
df.filter(df['Gender'].isNotNull()).count()

3823158

In [41]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [44]:
#write to csv with pyspark
# df.repartition(1).write.csv("it.csv", sep=',')
# !cp it.csv "drive/My Drive/"

In [45]:
# write to csv with pandas
# df.toPandas().to_csv('ppp_p.csv', index=0)
# !cp ppp_p.csv "drive/My Drive/"